In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
tqdm.pandas()

In [2]:
%%time
last_df = pd.read_csv('./Prepped Data/land_specs_last_baseline_20211116.csv')
print(last_df.shape)
last_df.head()

(1000126, 16)
Wall time: 2.14 s


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자
0,서울특별시 강남구 개포동,일반,100,5934,2021,1,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,424000,2021-07-29
1,서울특별시 강남구 개포동,일반,101,5935,2021,1,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29
2,서울특별시 강남구 개포동,일반,102,5936,2021,1,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29
3,서울특별시 강남구 개포동,일반,103,5937,2021,1,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지,351000,2021-07-29
4,서울특별시 강남구 개포동,일반,104-1,5938,2021,1,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지,422600,2021-07-29


In [3]:
last_df['본번'] = last_df['지번'].apply(lambda x: x.split('-')[0])

In [4]:
def 부번필터(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    splitted = x.split('-')
    
    if len(splitted) == 1:
        return '0'
    else:
        return splitted[-1]

In [5]:
last_df['부번'] = last_df['지번'].apply(부번필터)

In [6]:
last_df['토지일련번호'].nunique()

53689

In [7]:
%%time
addr_coor_df = pd.read_csv('./Prepped Data/addr_coor_baseline_20211126.csv')
print(addr_coor_df.shape)
addr_coor_df.head()

(539989, 14)
Wall time: 625 ms


,시도명,시군구명,읍면동명,산여부,지번본번(번지),지번부번(호),도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표
0,서울특별시,종로구,청운동,0,144,3,자하문로,0,94,0,근린생활시설,0,953241.683263,1.954023e+06
1,서울특별시,종로구,청운동,0,108,14,자하문로,0,96,0,주택,0,953243.258151,1.954034e+06
2,서울특별시,종로구,청운동,0,108,13,자하문로,0,98,0,주택,0,953240.877646,1.954055e+06
3,서울특별시,종로구,청운동,0,134,2,자하문로,0,99,3,주택,1,953195.459364,1.954079e+06
4,서울특별시,종로구,청운동,0,130,2,자하문로,0,99,4,주택,1,953193.346779,1.954097e+06


In [8]:
addr_coor_df = addr_coor_df.rename(columns={'지번본번(번지)':'본번', '지번부번(호)': '부번'})

In [9]:
last_df.head()

,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번
0,서울특별시 강남구 개포동,일반,100,5934,2021,1,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,424000,2021-07-29,100,0
1,서울특별시 강남구 개포동,일반,101,5935,2021,1,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29,101,0
2,서울특별시 강남구 개포동,일반,102,5936,2021,1,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29,102,0
3,서울특별시 강남구 개포동,일반,103,5937,2021,1,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지,351000,2021-07-29,103,0
4,서울특별시 강남구 개포동,일반,104-1,5938,2021,1,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지,422600,2021-07-29,104,1


In [10]:
addr_coor_df['법정동명'] = addr_coor_df['시도명'] + ' ' + addr_coor_df['시군구명'] + ' ' + addr_coor_df['읍면동명']

In [11]:
addr_coor_df.head()

,시도명,시군구명,읍면동명,산여부,본번,부번,도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표,법정동명
0,서울특별시,종로구,청운동,0,144,3,자하문로,0,94,0,근린생활시설,0,953241.683263,1.954023e+06,서울특별시 종로구 청운동
1,서울특별시,종로구,청운동,0,108,14,자하문로,0,96,0,주택,0,953243.258151,1.954034e+06,서울특별시 종로구 청운동
2,서울특별시,종로구,청운동,0,108,13,자하문로,0,98,0,주택,0,953240.877646,1.954055e+06,서울특별시 종로구 청운동
3,서울특별시,종로구,청운동,0,134,2,자하문로,0,99,3,주택,1,953195.459364,1.954079e+06,서울특별시 종로구 청운동
4,서울특별시,종로구,청운동,0,130,2,자하문로,0,99,4,주택,1,953193.346779,1.954097e+06,서울특별시 종로구 청운동


In [12]:
last_df['대장구분명'].value_counts()

일반             975557
산               17987
가지번              2799
블럭지번             1565
블럭지번(지구)         1348
기타지번              677
블럭지번(지구_롯트)       155
가지번(부번세분)          37
Name: 대장구분명, dtype: int64

In [13]:
addr_coor_df['산여부'].value_counts()

0    537608
1      2381
Name: 산여부, dtype: int64

In [14]:
addr_coor_df['도로명'].unique()[:100]

array(['자하문로', '창의문로', '자하문로26길', '자하문로28가길', '자하문로28길', '자하문로30길',
       '자하문로33가길', '자하문로33길', '자하문로33나길', '자하문로33다길', '자하문로35길',
       '자하문로36길', '창의문로1길', '필운대로', '옥인5길', '자하문로21길', '자하문로23길',
       '필운대로10길', '필운대로11길', '필운대로13길', '효자로', '자하문로24길', '자하문로16길',
       '효자로13길', '자하문로10길', '자하문로12길', '자하문로14길', '효자로9길', '자하문로2길',
       '자하문로4길', '자하문로6길', '자하문로8길', '효자로7길', '사직로', '사직로8길', '새문안로5가길',
       '새문안로5길', '자하문로11길', '자하문로13길', '자하문로15길', '자하문로17길', '자하문로7길',
       '자하문로9길', '필운대로6길', '옥인2길', '옥인3길', '옥인6가길', '옥인6길', '옥인길',
       '필운대로5가길', '필운대로5나길', '필운대로5다길', '필운대로2길', '필운대로3길', '필운대로5길',
       '인왕산로', '옥인1길', '옥인4길', '자하문로19길', '필운대로7길', '필운대로9가길', '필운대로9길',
       '필운대로9나길', '자하문로1가길', '자하문로1길', '자하문로1나길', '자하문로1다길', '자하문로1라길',
       '자하문로5가길', '자하문로5길', '사직로10길', '사직로9가길', '사직로9길', '필운대로1길',
       '사직로12길', '경희궁3가길', '경희궁3길', '경희궁3나길', '경희궁길', '사직로5길', '사직로6길',
       '사직로7길', '송월1길', '인왕산로1길', '새문안로9길', '세종대로23길', '새문안로', '새문안로3길',
       '새문안로7길', '경희궁2길',

In [15]:
inv_df = last_df[last_df['대장구분명'] == '블럭지번']
print(inv_df.shape)
inv_df.head()

(1565, 18)


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번
16125,서울특별시 강남구 세곡동,블럭지번,1,999999,2013,1,학교용지,11814.0,개발제한구역,자연녹지지역,주거기타,평지,사다리형,광대로한면,1000000,2017-10-17,1,0
16150,서울특별시 강남구 세곡동,블럭지번,10,999999,2013,1,대,251.0,제1종일반주거지역,지정되지않음,주거나지,평지,정방형,소로각지,1800000,2017-10-17,10,0
16153,서울특별시 강남구 세곡동,블럭지번,10-1,25911,2013,1,대,250.0,제1종일반주거지역,지정되지않음,주거나지,평지,세로장방,소로한면,1720000,2017-10-17,10,1
16154,서울특별시 강남구 세곡동,블럭지번,10-10,25920,2013,1,대,255.0,제1종일반주거지역,지정되지않음,주거나지,평지,세로장방,소로한면,1720000,2017-10-17,10,10
16155,서울특별시 강남구 세곡동,블럭지번,10-11,25921,2013,1,대,255.0,제1종일반주거지역,지정되지않음,주거나지,평지,세로장방,소로한면,1720000,2017-10-17,10,11


In [16]:
last_df['산여부'] = last_df['대장구분명'].apply(lambda x: 1 if x == '산' else 0)

In [17]:
last_df['산여부'].value_counts()

0    982139
1     17987
Name: 산여부, dtype: int64

In [18]:
last_df.head()

,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,산여부
0,서울특별시 강남구 개포동,일반,100,5934,2021,1,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,424000,2021-07-29,100,0,0
1,서울특별시 강남구 개포동,일반,101,5935,2021,1,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29,101,0,0
2,서울특별시 강남구 개포동,일반,102,5936,2021,1,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29,102,0,0
3,서울특별시 강남구 개포동,일반,103,5937,2021,1,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지,351000,2021-07-29,103,0,0
4,서울특별시 강남구 개포동,일반,104-1,5938,2021,1,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지,422600,2021-07-29,104,1,0


In [19]:
addr_coor_df.head()

,시도명,시군구명,읍면동명,산여부,본번,부번,도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표,법정동명
0,서울특별시,종로구,청운동,0,144,3,자하문로,0,94,0,근린생활시설,0,953241.683263,1.954023e+06,서울특별시 종로구 청운동
1,서울특별시,종로구,청운동,0,108,14,자하문로,0,96,0,주택,0,953243.258151,1.954034e+06,서울특별시 종로구 청운동
2,서울특별시,종로구,청운동,0,108,13,자하문로,0,98,0,주택,0,953240.877646,1.954055e+06,서울특별시 종로구 청운동
3,서울특별시,종로구,청운동,0,134,2,자하문로,0,99,3,주택,1,953195.459364,1.954079e+06,서울특별시 종로구 청운동
4,서울특별시,종로구,청운동,0,130,2,자하문로,0,99,4,주택,1,953193.346779,1.954097e+06,서울특별시 종로구 청운동


In [20]:
addr_coor_df['본번'] = addr_coor_df['본번'].apply(str)
addr_coor_df['부번'] = addr_coor_df['부번'].apply(str)

In [21]:
last_df['산여부'].dtype

dtype('int64')

In [22]:
addr_coor_df['산여부'].dtype

dtype('int64')

In [23]:
addr_coor_df['건물용도분류'].value_counts()

주택                                       395067
근린생활시설                                    91045
업무시설                                      20003
교육및복지시설                                    6031
유통시설                                       5224
종교시설                                       4397
공장/창고시설                                    3276
숙박시설                                       2271
의료시설                                       2022
자동차관련시설                                    1773
주택,근린생활시설                                  1393
문화/관광/레저시설                                 1053
보안/위험시설                                     561
여객(화물)운송시설                                  226
근린생활시설,업무시설                                 207
공공용시설                                       194
환경정화시설                                      188
주택,업무시설                                     175
근린생활시설,공장/창고시설                              172
주택,공장/창고시설                                  167
유흥/위락시설                                 

In [24]:
addr_coor_df['본번'] = addr_coor_df['본번'].apply(int)
addr_coor_df['부번'] = addr_coor_df['부번'].apply(int)

In [25]:
addr_coor_df = addr_coor_df.sort_values(['법정동명', '산여부', '본번', '부번', '도로명', '지하여부', '건물본번', '건물부번']).reset_index(drop=True)
print(addr_coor_df.shape)
addr_coor_df.head()

(539989, 15)


,시도명,시군구명,읍면동명,산여부,본번,부번,도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표,법정동명
0,서울특별시,강남구,개포동,0,12,0,개포로109길,0,5,0,"주택,교육및복지시설",1,962506.305341,1.943834e+06,서울특별시 강남구 개포동
1,서울특별시,강남구,개포동,0,12,0,개포로109길,0,9,0,주택,1,962515.184710,1.943854e+06,서울특별시 강남구 개포동
2,서울특별시,강남구,개포동,0,12,0,개포로109길,0,21,0,주택,1,962572.791839,1.943970e+06,서울특별시 강남구 개포동
3,서울특별시,강남구,개포동,0,12,2,개포로109길,0,69,0,주택,1,962641.068306,1.944198e+06,서울특별시 강남구 개포동
4,서울특별시,강남구,개포동,0,12,3,남부순환로,0,3142,30,근린생활시설,0,962480.314302,1.944213e+06,서울특별시 강남구 개포동


In [26]:
addr_coor_df['지번평균_x좌표'] = addr_coor_df[['법정동명', '산여부', '본번', '부번', 'X좌표']].groupby(['법정동명',
                                                                                            '산여부', '본번', '부번'
                                                                                           ]).transform('mean')['X좌표']

In [27]:
addr_coor_df['지번평균_y좌표'] = addr_coor_df[['법정동명', '산여부', '본번', '부번', 'Y좌표']].groupby(['법정동명',
                                                                                            '산여부', '본번', '부번'
                                                                                           ]).transform('mean')['Y좌표']

In [28]:
addr_coor_df.head()

,시도명,시군구명,읍면동명,산여부,본번,부번,도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표,법정동명,지번평균_x좌표,지번평균_y좌표
0,서울특별시,강남구,개포동,0,12,0,개포로109길,0,5,0,"주택,교육및복지시설",1,962506.305341,1.943834e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06
1,서울특별시,강남구,개포동,0,12,0,개포로109길,0,9,0,주택,1,962515.184710,1.943854e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06
2,서울특별시,강남구,개포동,0,12,0,개포로109길,0,21,0,주택,1,962572.791839,1.943970e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06
3,서울특별시,강남구,개포동,0,12,2,개포로109길,0,69,0,주택,1,962641.068306,1.944198e+06,서울특별시 강남구 개포동,962641.068306,1.944198e+06
4,서울특별시,강남구,개포동,0,12,3,남부순환로,0,3142,30,근린생활시설,0,962480.314302,1.944213e+06,서울특별시 강남구 개포동,962480.314302,1.944213e+06


In [29]:
addr_coor_df['지번주소'] = addr_coor_df['법정동명'] + ' ' + addr_coor_df['본번'].apply(str) + '-' + addr_coor_df['부번'].apply(str)
print(addr_coor_df.shape)
addr_coor_df.head()

(539989, 18)


,시도명,시군구명,읍면동명,산여부,본번,부번,도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표,법정동명,지번평균_x좌표,지번평균_y좌표,지번주소
0,서울특별시,강남구,개포동,0,12,0,개포로109길,0,5,0,"주택,교육및복지시설",1,962506.305341,1.943834e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0
1,서울특별시,강남구,개포동,0,12,0,개포로109길,0,9,0,주택,1,962515.184710,1.943854e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0
2,서울특별시,강남구,개포동,0,12,0,개포로109길,0,21,0,주택,1,962572.791839,1.943970e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0
3,서울특별시,강남구,개포동,0,12,2,개포로109길,0,69,0,주택,1,962641.068306,1.944198e+06,서울특별시 강남구 개포동,962641.068306,1.944198e+06,서울특별시 강남구 개포동 12-2
4,서울특별시,강남구,개포동,0,12,3,남부순환로,0,3142,30,근린생활시설,0,962480.314302,1.944213e+06,서울특별시 강남구 개포동,962480.314302,1.944213e+06,서울특별시 강남구 개포동 12-3


In [30]:
for addr in tqdm(addr_coor_df['지번주소'].unique()):
    addr_df = addr_coor_df[addr_coor_df['지번주소'] == addr]
    
    if len(addr_df['건물군여부'].unique()) > 1:
        print(addr_df.shape)
        display(addr_df)
        
        break

  0%|                                                                            | 24/523126 [00:00<3:14:12, 44.89it/s]

(2, 18)


,시도명,시군구명,읍면동명,산여부,본번,부번,도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표,법정동명,지번평균_x좌표,지번평균_y좌표,지번주소
26,서울특별시,강남구,개포동,0,155,0,개포로,0,408,0,교육및복지시설,0,961326.391741,1.942997e+06,서울특별시 강남구 개포동,961313.426683,1.943083e+06,서울특별시 강남구 개포동 155-0
27,서울특별시,강남구,개포동,0,155,0,개포로,0,410,0,교육및복지시설,1,961300.461625,1.943169e+06,서울특별시 강남구 개포동,961313.426683,1.943083e+06,서울특별시 강남구 개포동 155-0


  0%|                                                                            | 24/523126 [00:00<3:32:33, 41.02it/s]


In [31]:
addr_coor_df['건물갯수'] = addr_coor_df[['법정동명', '산여부', '본번', '부번', '건물용도분류']].groupby(['법정동명', '산여부', '본번', '부번']).transform('count')['건물용도분류']

In [32]:
addr_coor_df['건물용도분류'].mode()[0]

'주택'

In [33]:
%%time
addr_coor_df['최빈건물용도'] = addr_coor_df[['법정동명', '산여부', '본번', '부번', '건물용도분류']].groupby(['법정동명', '산여부', '본번', '부번']).transform(lambda x: x.value_counts()[0])['건물용도분류']

IndexError: index 0 is out of bounds for axis 0 with size 0

In [34]:
addr_coor_df['건물용도분류'] = addr_coor_df['건물용도분류'].fillna('입력되지않음')

In [35]:
addr_coor_df['건물용도분류'].isna().sum()

0

In [36]:
addr_coor_df['건물용도분류'].mode()[0]

'주택'

In [37]:
addr_coor_df['건물용도분류'].value_counts().index[-1]

'숙박시설,공공용시설'

In [38]:
%%time
addr_coor_df['최빈건물용도'] = addr_coor_df[['법정동명', '산여부', '본번', '부번', '건물용도분류']].groupby(['법정동명', '산여부', '본번', '부번']).progress_transform(lambda x: x.mode()[0])['건물용도분류']

100%|████████████████████████████████████████████████████████████████████████| 523223/523223 [07:50<00:00, 1111.32it/s]

Wall time: 7min 51s


In [39]:
%%time
addr_coor_df['least_frequent_usage'] = addr_coor_df[['법정동명', '산여부', '본번', '부번', '건물용도분류']].groupby(['법정동명', '산여부', '본번', '부번']).progress_transform(lambda x: x.value_counts().index[-1])['건물용도분류']

100%|█████████████████████████████████████████████████████████████████████████| 523223/523223 [11:16<00:00, 772.99it/s]

Wall time: 11min 17s


In [40]:
addr_coor_df.head()

,시도명,시군구명,읍면동명,산여부,본번,부번,도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표,법정동명,지번평균_x좌표,지번평균_y좌표,지번주소,건물갯수,최빈건물용도,least_frequent_usage
0,서울특별시,강남구,개포동,0,12,0,개포로109길,0,5,0,"주택,교육및복지시설",1,962506.305341,1.943834e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
1,서울특별시,강남구,개포동,0,12,0,개포로109길,0,9,0,주택,1,962515.184710,1.943854e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
2,서울특별시,강남구,개포동,0,12,0,개포로109길,0,21,0,주택,1,962572.791839,1.943970e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
3,서울특별시,강남구,개포동,0,12,2,개포로109길,0,69,0,주택,1,962641.068306,1.944198e+06,서울특별시 강남구 개포동,962641.068306,1.944198e+06,서울특별시 강남구 개포동 12-2,1,주택,주택
4,서울특별시,강남구,개포동,0,12,3,남부순환로,0,3142,30,근린생활시설,0,962480.314302,1.944213e+06,서울특별시 강남구 개포동,962480.314302,1.944213e+06,서울특별시 강남구 개포동 12-3,1,근린생활시설,근린생활시설


In [41]:
addr_coor_df = addr_coor_df.rename(columns={'건물갯수':'지번별_도로명주소갯수'})
print(addr_coor_df.shape)
addr_coor_df.head()

(539989, 21)


,시도명,시군구명,읍면동명,산여부,본번,부번,도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표,법정동명,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시,강남구,개포동,0,12,0,개포로109길,0,5,0,"주택,교육및복지시설",1,962506.305341,1.943834e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
1,서울특별시,강남구,개포동,0,12,0,개포로109길,0,9,0,주택,1,962515.184710,1.943854e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
2,서울특별시,강남구,개포동,0,12,0,개포로109길,0,21,0,주택,1,962572.791839,1.943970e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
3,서울특별시,강남구,개포동,0,12,2,개포로109길,0,69,0,주택,1,962641.068306,1.944198e+06,서울특별시 강남구 개포동,962641.068306,1.944198e+06,서울특별시 강남구 개포동 12-2,1,주택,주택
4,서울특별시,강남구,개포동,0,12,3,남부순환로,0,3142,30,근린생활시설,0,962480.314302,1.944213e+06,서울특별시 강남구 개포동,962480.314302,1.944213e+06,서울특별시 강남구 개포동 12-3,1,근린생활시설,근린생활시설


In [42]:
print(last_df.shape)
last_df.head()

(1000126, 19)


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,산여부
0,서울특별시 강남구 개포동,일반,100,5934,2021,1,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,424000,2021-07-29,100,0,0
1,서울특별시 강남구 개포동,일반,101,5935,2021,1,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29,101,0,0
2,서울특별시 강남구 개포동,일반,102,5936,2021,1,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29,102,0,0
3,서울특별시 강남구 개포동,일반,103,5937,2021,1,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지,351000,2021-07-29,103,0,0
4,서울특별시 강남구 개포동,일반,104-1,5938,2021,1,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지,422600,2021-07-29,104,1,0


In [43]:
cols_to_test = ['시도명', '시군구명', '읍면동명']

for col in cols_to_test:
    print(addr_coor_df[col].isna().sum())
    print(addr_coor_df[col].str.contains(' ').sum())

0
0
0
0
0
0


In [44]:
addr_coor_df['법정동명'] = addr_coor_df['시도명'] + ' ' + addr_coor_df['시군구명'] + ' ' + addr_coor_df['읍면동명']

In [45]:
addr_coor_df['본번'] = addr_coor_df['본번'].apply(str)
addr_coor_df['부번'] = addr_coor_df['부번'].apply(str)

In [46]:
last_df['본번'] = last_df['본번'].apply(str)
last_df['부번'] = last_df['부번'].apply(str)

In [47]:
last_df.head()

,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,산여부
0,서울특별시 강남구 개포동,일반,100,5934,2021,1,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,424000,2021-07-29,100,0,0
1,서울특별시 강남구 개포동,일반,101,5935,2021,1,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29,101,0,0
2,서울특별시 강남구 개포동,일반,102,5936,2021,1,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,513000,2021-07-29,102,0,0
3,서울특별시 강남구 개포동,일반,103,5937,2021,1,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지,351000,2021-07-29,103,0,0
4,서울특별시 강남구 개포동,일반,104-1,5938,2021,1,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지,422600,2021-07-29,104,1,0


In [48]:
addr_coor_df.head()

,시도명,시군구명,읍면동명,산여부,본번,부번,도로명,지하여부,건물본번,건물부번,건물용도분류,건물군여부,X좌표,Y좌표,법정동명,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시,강남구,개포동,0,12,0,개포로109길,0,5,0,"주택,교육및복지시설",1,962506.305341,1.943834e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
1,서울특별시,강남구,개포동,0,12,0,개포로109길,0,9,0,주택,1,962515.184710,1.943854e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
2,서울특별시,강남구,개포동,0,12,0,개포로109길,0,21,0,주택,1,962572.791839,1.943970e+06,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
3,서울특별시,강남구,개포동,0,12,2,개포로109길,0,69,0,주택,1,962641.068306,1.944198e+06,서울특별시 강남구 개포동,962641.068306,1.944198e+06,서울특별시 강남구 개포동 12-2,1,주택,주택
4,서울특별시,강남구,개포동,0,12,3,남부순환로,0,3142,30,근린생활시설,0,962480.314302,1.944213e+06,서울특별시 강남구 개포동,962480.314302,1.944213e+06,서울특별시 강남구 개포동 12-3,1,근린생활시설,근린생활시설


In [49]:
addr_coor_df = addr_coor_df.drop(columns=['도로명', '지하여부', '건물본번', '건물부번', '건물용도분류', 'X좌표', 'Y좌표']).drop_duplicates().reset_index(drop=True)
print(addr_coor_df.shape)
addr_coor_df.head()

(525103, 14)


,시도명,시군구명,읍면동명,산여부,본번,부번,건물군여부,법정동명,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시,강남구,개포동,0,12,0,1,서울특별시 강남구 개포동,962531.427297,1.943886e+06,서울특별시 강남구 개포동 12-0,3,주택,"주택,교육및복지시설"
1,서울특별시,강남구,개포동,0,12,2,1,서울특별시 강남구 개포동,962641.068306,1.944198e+06,서울특별시 강남구 개포동 12-2,1,주택,주택
2,서울특별시,강남구,개포동,0,12,3,0,서울특별시 강남구 개포동,962480.314302,1.944213e+06,서울특별시 강남구 개포동 12-3,1,근린생활시설,근린생활시설
3,서울특별시,강남구,개포동,0,12,4,0,서울특별시 강남구 개포동,962492.091516,1.943790e+06,서울특별시 강남구 개포동 12-4,1,유통시설,유통시설
4,서울특별시,강남구,개포동,0,12,5,0,서울특별시 강남구 개포동,962449.640864,1.943776e+06,서울특별시 강남구 개포동 12-5,1,교육및복지시설,교육및복지시설


In [50]:
merge_df = last_df.merge(addr_coor_df, on=['법정동명', '산여부', '본번', '부번'])
print(merge_df.shape)
merge_df.head()

(523964, 29)


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,산여부,시도명,시군구명,읍면동명,건물군여부,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시 강남구 개포동,일반,1055,6899,2021,1,대,3017.0,자연녹지지역,지정되지않음,주거기타,완경사,사다리형,광대로한면,2248000,2021-07-29,1055,0,0,서울특별시,강남구,개포동,0,960424.632847,1.941490e+06,서울특별시 강남구 개포동 1055-0,1,종교시설,종교시설
1,서울특별시 강남구 개포동,일반,1057-9,6908,2021,1,대,3062.0,개발제한구역,자연녹지지역,주거기타,고지,사다리형,광대로한면,1890000,2021-07-29,1057,9,0,서울특별시,강남구,개포동,0,960301.686654,1.941361e+06,서울특별시 강남구 개포동 1057-9,0,입력되지않음,입력되지않음
2,서울특별시 강남구 개포동,일반,108-1,5946,2021,1,전,24.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,706300,2021-07-29,108,1,0,서울특별시,강남구,개포동,0,962252.424116,1.942756e+06,서울특별시 강남구 개포동 108-1,1,주택,주택
3,서울특별시 강남구 개포동,일반,1163,6918,2021,1,대,335.8,제3종일반주거지역,지정되지않음,상업용,평지,세로장방,광대세각,10700000,2021-07-29,1163,0,0,서울특별시,강남구,개포동,0,960207.540757,1.941608e+06,서울특별시 강남구 개포동 1163-0,1,업무시설,업무시설
4,서울특별시 강남구 개포동,일반,1163-1,6919,2021,1,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,10700000,2021-07-29,1163,1,0,서울특별시,강남구,개포동,0,960234.133087,1.941599e+06,서울특별시 강남구 개포동 1163-1,1,자동차관련시설,자동차관련시설


In [51]:
merge_df.to_csv('./Prepped Data/last_addr_coor_merge_baseline_20211129.csv', index=False)